# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f2a2e2b78b0>
├── 'a' --> tensor([[ 0.5781,  0.7896, -0.2249],
│                   [ 0.1723,  2.0271, -0.1867]])
└── 'x' --> <FastTreeValue 0x7f2a2e2b7a90>
    └── 'c' --> tensor([[ 0.3466,  1.4310, -0.8847, -0.2101],
                        [ 0.7148,  1.0139,  0.0217, -0.6997],
                        [ 0.7825, -0.0507,  0.2250, -0.2242]])

In [4]:
t.a

tensor([[ 0.5781,  0.7896, -0.2249],
        [ 0.1723,  2.0271, -0.1867]])

In [5]:
%timeit t.a

70.7 ns ± 1.85 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f2a2e2b78b0>
├── 'a' --> tensor([[ 0.8544, -2.6020, -0.2738],
│                   [-0.9298,  0.1250,  1.1306]])
└── 'x' --> <FastTreeValue 0x7f2a2e2b7a90>
    └── 'c' --> tensor([[ 0.3466,  1.4310, -0.8847, -0.2101],
                        [ 0.7148,  1.0139,  0.0217, -0.6997],
                        [ 0.7825, -0.0507,  0.2250, -0.2242]])

In [7]:
%timeit t.a = new_value

92 ns ± 1.19 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.5781,  0.7896, -0.2249],
               [ 0.1723,  2.0271, -0.1867]]),
    x: Batch(
           c: tensor([[ 0.3466,  1.4310, -0.8847, -0.2101],
                      [ 0.7148,  1.0139,  0.0217, -0.6997],
                      [ 0.7825, -0.0507,  0.2250, -0.2242]]),
       ),
)

In [10]:
b.a

tensor([[ 0.5781,  0.7896, -0.2249],
        [ 0.1723,  2.0271, -0.1867]])

In [11]:
%timeit b.a

62.7 ns ± 2.46 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.1307, -0.4304, -0.4552],
               [-0.4090,  1.6508, -0.5903]]),
    x: Batch(
           c: tensor([[ 0.3466,  1.4310, -0.8847, -0.2101],
                      [ 0.7148,  1.0139,  0.0217, -0.6997],
                      [ 0.7825, -0.0507,  0.2250, -0.2242]]),
       ),
)

In [13]:
%timeit b.a = new_value

576 ns ± 24.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

946 ns ± 13.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.2 µs ± 521 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

278 µs ± 5.93 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

262 µs ± 8.94 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f2974b604f0>
├── 'a' --> tensor([[[ 0.5781,  0.7896, -0.2249],
│                    [ 0.1723,  2.0271, -0.1867]],
│           
│                   [[ 0.5781,  0.7896, -0.2249],
│                    [ 0.1723,  2.0271, -0.1867]],
│           
│                   [[ 0.5781,  0.7896, -0.2249],
│                    [ 0.1723,  2.0271, -0.1867]],
│           
│                   [[ 0.5781,  0.7896, -0.2249],
│                    [ 0.1723,  2.0271, -0.1867]],
│           
│                   [[ 0.5781,  0.7896, -0.2249],
│                    [ 0.1723,  2.0271, -0.1867]],
│           
│                   [[ 0.5781,  0.7896, -0.2249],
│                    [ 0.1723,  2.0271, -0.1867]],
│           
│                   [[ 0.5781,  0.7896, -0.2249],
│                    [ 0.1723,  2.0271, -0.1867]],
│           
│                   [[ 0.5781,  0.7896, -0.2249],
│                    [ 0.1723,  2.0271, -0.1867]]])
└── 'x' --> <FastTreeValue 0x7f297b4d14c0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

45.5 µs ± 1.24 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f29744c39a0>
├── 'a' --> tensor([[ 0.5781,  0.7896, -0.2249],
│                   [ 0.1723,  2.0271, -0.1867],
│                   [ 0.5781,  0.7896, -0.2249],
│                   [ 0.1723,  2.0271, -0.1867],
│                   [ 0.5781,  0.7896, -0.2249],
│                   [ 0.1723,  2.0271, -0.1867],
│                   [ 0.5781,  0.7896, -0.2249],
│                   [ 0.1723,  2.0271, -0.1867],
│                   [ 0.5781,  0.7896, -0.2249],
│                   [ 0.1723,  2.0271, -0.1867],
│                   [ 0.5781,  0.7896, -0.2249],
│                   [ 0.1723,  2.0271, -0.1867],
│                   [ 0.5781,  0.7896, -0.2249],
│                   [ 0.1723,  2.0271, -0.1867],
│                   [ 0.5781,  0.7896, -0.2249],
│                   [ 0.1723,  2.0271, -0.1867]])
└── 'x' --> <FastTreeValue 0x7f297b0eb820>
    └── 'c' --> tensor([[ 0.3466,  1.4310, -0.8847, -0.2101],
                        [ 0.7148,  1.0139,  0.0217, -0.6997],
                 

In [23]:
%timeit t_cat(trees)

42 µs ± 1.11 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

81.7 µs ± 1.28 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.3466,  1.4310, -0.8847, -0.2101],
                       [ 0.7148,  1.0139,  0.0217, -0.6997],
                       [ 0.7825, -0.0507,  0.2250, -0.2242]],
              
                      [[ 0.3466,  1.4310, -0.8847, -0.2101],
                       [ 0.7148,  1.0139,  0.0217, -0.6997],
                       [ 0.7825, -0.0507,  0.2250, -0.2242]],
              
                      [[ 0.3466,  1.4310, -0.8847, -0.2101],
                       [ 0.7148,  1.0139,  0.0217, -0.6997],
                       [ 0.7825, -0.0507,  0.2250, -0.2242]],
              
                      [[ 0.3466,  1.4310, -0.8847, -0.2101],
                       [ 0.7148,  1.0139,  0.0217, -0.6997],
                       [ 0.7825, -0.0507,  0.2250, -0.2242]],
              
                      [[ 0.3466,  1.4310, -0.8847, -0.2101],
                       [ 0.7148,  1.0139,  0.0217, -0.6997],
                       [ 0.7825, -0.0507,  0.2250, -0.2242]],

In [26]:
%timeit Batch.stack(batches)

102 µs ± 2.64 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.3466,  1.4310, -0.8847, -0.2101],
                      [ 0.7148,  1.0139,  0.0217, -0.6997],
                      [ 0.7825, -0.0507,  0.2250, -0.2242],
                      [ 0.3466,  1.4310, -0.8847, -0.2101],
                      [ 0.7148,  1.0139,  0.0217, -0.6997],
                      [ 0.7825, -0.0507,  0.2250, -0.2242],
                      [ 0.3466,  1.4310, -0.8847, -0.2101],
                      [ 0.7148,  1.0139,  0.0217, -0.6997],
                      [ 0.7825, -0.0507,  0.2250, -0.2242],
                      [ 0.3466,  1.4310, -0.8847, -0.2101],
                      [ 0.7148,  1.0139,  0.0217, -0.6997],
                      [ 0.7825, -0.0507,  0.2250, -0.2242],
                      [ 0.3466,  1.4310, -0.8847, -0.2101],
                      [ 0.7148,  1.0139,  0.0217, -0.6997],
                      [ 0.7825, -0.0507,  0.2250, -0.2242],
                      [ 0.3466,  1.4310, -0.8847, -0.2101],
                   

In [28]:
%timeit Batch.cat(batches)

175 µs ± 3.62 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

539 µs ± 9.33 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
